In [1]:
import math
import numpy as np
import random
from collections import Counter
from scipy import stats
from tqdm import tqdm
from scipy import stats

In [ ]:
from os import walk
mypath = '../data/'
filenames = next(walk(mypath), (None, None, []))[2] 

NUM_DIGITS = 1000000
NUM_RUNS = len(pi_cf_digits)//NUM_DIGITS

pi_cf_digits = []
for filename in tqdm(filenames):
    pi_digits = np.load(filename)
    pi_cf_digits.extend(pi_digits['arr_0'])
pi_cf_digits = pi_cf_digits[:math.floor((len(pi_cf_digits)/NUM_DIGITS))* NUM_DIGITS]
pi_cf_digits = [int(x) for x in pi_cf_digits]
len(pi_cf_digits)

# Normalized distance

### Random Walk

In [13]:
nums = [math.log(1+1/(k*(k+2)))/math.log(2) for k in range(1, NUM_DIGITS+1)]
distribution = [0 for i in range(4)]
for i in range(len(nums)):
    distribution[i%4] += nums[i]
inv_distribution = [1/distribution[i] for i in range(len(distribution))]
normalized_inv_distribution = [float(i)/sum(inv_distribution) for i in inv_distribution]
moves = [(1*inv_distribution[0], 0),
         (0, 1*inv_distribution[1]),
         (-1*inv_distribution[2], 0),
         (0, -1*inv_distribution[3])]

In [14]:
moves

[(2.000001442695982, 0),
 (0, 4.18055196959628),
 (-6.600844419128332, 0),
 (0, -9.149106876384758)]

In [15]:
distribution

[0.49999963932626473,
 0.23920286298858545,
 0.15149576879923704,
 0.10930028619308763]

In [16]:
def dist_from_origin(coordinates):
    return math.sqrt((coordinates[0]**2) + (coordinates[1]**2))

In [17]:
def single_random_walk_run(random_dist):
    random_walk = [(0, 0)]
    for i in rand_distribution:
        random_walk.append((random_walk[-1][0] + moves[i%4][0], random_walk[-1][1] + moves[i%4][1]))
    distance = 0
    for i in random_walk[1:]:
        distance += dist_from_origin(i)
    avg_dist = distance/(NUM_DIGITS * 500*math.pi)
    return avg_dist

In [18]:
avg_distances = []
for i in tqdm(range(NUM_RUNS)):
    rand_distribution = random.choices([0, 1, 2, 3], weights = distribution, k = NUM_DIGITS)
    avg_distances.append(single_random_walk_run(rand_distribution))

100%|███████████████████████████████████████| 1101/1101 [08:42<00:00,  2.11it/s]


In [19]:
stats.describe(avg_distances)

DescribeResult(nobs=1101, minmax=(0.4532371371916253, 5.231803993840657), mean=1.7857855380243446, variance=0.518788239987269, skewness=0.8994600851328021, kurtosis=1.0866449903814228)

### $\pi$ Walk

In [20]:
groups = list(np.array_split(pi_cf_digits, NUM_RUNS))

In [21]:
def pi_walk(pi_digits):
    mod_4_digits = [(i-1)%4 for i in pi_digits]
    pi_walk = [(0, 0)]
    for i in range(len(mod_4_digits)):
        pi_walk.append((pi_walk[-1][0] + moves[mod_4_digits[i]][0], pi_walk[-1][1] + moves[mod_4_digits[i]][1]))
    total_distance = 0
    for i in range(len(pi_walk)):
        total_distance += dist_from_origin(pi_walk[i])
    return total_distance/(NUM_DIGITS * 500*math.pi)

In [22]:
avg_pi_distances = []
test = []
for i in tqdm(range(NUM_RUNS)):
    avg_pi_distances.append(pi_walk(groups[i]))

100%|███████████████████████████████████████| 1101/1101 [11:33<00:00,  1.59it/s]


In [23]:
stats.describe(avg_pi_distances)

DescribeResult(nobs=1101, minmax=(0.5824486592593773, 4.34641277917184), mean=1.7404330248048623, variance=0.45385931195668205, skewness=0.7374880356861712, kurtosis=0.19689123911785256)

# Sites visited

### Random Walk

In [24]:
nums = [math.log(1+1/(k*(k+2)))/math.log(2) for k in range(1, NUM_DIGITS+1)]
distribution = [0 for i in range(4)]
for i in range(len(nums)):
    distribution[i%4] += nums[i]
inv_distribution = [1/distribution[i] for i in range(len(distribution))]
normalized_inv_distribution = [float(i)/np.average(inv_distribution) for i in inv_distribution]
moves = [(1*normalized_inv_distribution[0], 0),
         (0, 1*normalized_inv_distribution[1]),
         (-1*normalized_inv_distribution[2], 0),
         (0, -1*normalized_inv_distribution[3])]

In [25]:
moves

[(0.3647889493367027, 0),
 (0, 0.7625090302838979),
 (-1.2039566817226974, 0),
 (0, -1.6687453386567015)]

In [26]:
def single_random_walk_run(random_dist):
    random_walk = [(0, 0)]
    for i in rand_distribution:
        random_walk.append((random_walk[-1][0] + moves[i%4][0], random_walk[-1][1] + moves[i%4][1]))
    sites = set()
    for i in random_walk:
        sites.add((math.floor(i[0]), math.floor(i[1])))
    return len(sites)

In [27]:
sites_visited = []
for i in tqdm(range(NUM_RUNS)):
    rand_distribution = random.choices([0, 1, 2, 3], weights = distribution, k = NUM_DIGITS)
    sites_visited.append(single_random_walk_run(rand_distribution))

100%|███████████████████████████████████████| 1101/1101 [11:40<00:00,  1.57it/s]


In [28]:
# sum
stats.describe(sites_visited)

DescribeResult(nobs=1101, minmax=(103471, 178590), mean=148008.5676657584, variance=141171260.82018992, skewness=-0.46802186120354655, kurtosis=0.3069269027007637)

In [29]:
# random_groups = []
# for i in range(NUM_RUNS):
#     random_groups.append(random.choices([0, 1, 2, 3], weights = distribution, k = NUM_DIGITS))

In [30]:
# start = time.time()
# sites_visited_v2 = Parallel(n_jobs=3)(delayed(run)(random_groups[i]) for i in range(NUM_RUNS))
# print(time.time() - start)

In [31]:
# stats.describe(sites_visited_v2)

### $\pi$ Walk

In [32]:
groups = list(np.array_split(pi_cf_digits, NUM_RUNS))

In [33]:
def pi_walk(pi_digits):
    mod_4_digits = [(i-1)%4 for i in pi_digits]
    pi_walk = [(0, 0)]
    for i in range(len(mod_4_digits)):
        pi_walk.append((pi_walk[-1][0] + moves[mod_4_digits[i]][0], pi_walk[-1][1] + moves[mod_4_digits[i]][1]))
    sites = set()
    for i in pi_walk:
        sites.add((math.floor(i[0]), math.floor(i[1])))
    return len(sites)

In [34]:
sites_visited_pi = []
for i in tqdm(range(NUM_RUNS)):
    sites_visited_pi.append(pi_walk(groups[i]))

100%|███████████████████████████████████████| 1101/1101 [14:12<00:00,  1.29it/s]


In [35]:
stats.describe(sites_visited_pi)

DescribeResult(nobs=1101, minmax=(96554, 176615), mean=146285.92915531335, variance=135363112.16043103, skewness=-0.5247485873879998, kurtosis=0.35934094714251863)

In [ ]:
# DescribeResult(nobs=1000, minmax=(7065, 15616),             
#                mean=11181.698, variance=1642525.7985945947,                
#                skewness=-0.21542613565347027, kurtosis=-0.17792074227761923)